## Imports and drive mount

In [ ]:
!pip install datasets
!pip install torchinfo
!pip install fasttext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... 

Creamos la carpeta model_storage

In [ ]:
!mkdir model_storage

In [ ]:
import pandas as pd
import numpy as np
import json
from collections import Counter
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary

from transformers import AutoTokenizer, AutoModelForMaskedLM

from datasets import DatasetDict, load_from_disk

from argparse import Namespace
import fasttext.util

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## TF - IDF

Path para cargar los datasets

In [ ]:
# Path selection
# file_path_gl = '/content/drive/My Drive/4º Carrera/NLP/Proyecto/gl_dataset.csv'  # PATH: Luis
file_path_gl = '/content/drive/MyDrive/4.Curso/NLP/Proyecto/gl_dataset.csv'  # PATH: Iker

# file_path_pt = '/content/drive/My Drive/4º Carrera/NLP/Proyecto/pt_dataset.csv'  # PATH: Luis
file_path_pt = '/content/drive/MyDrive/4.Curso/NLP/Proyecto/pt_dataset.csv'  # PATH: Iker

Utilizando ambos datasets al completo

In [ ]:
# Cargar ambos datasets
df_pt = pd.read_csv(file_path_pt)
df_gl = pd.read_csv(file_path_gl)

# Combinar los datasets
df_combined = pd.concat([df_pt, df_gl], ignore_index=True)

# Verificar resultados
print(f"Portugués: {len(df_pt)} filas seleccionadas")
print(f"Gallego: {len(df_gl)} filas seleccionadas")
print(f"Total combinado: {len(df_combined)} filas")

Portugués: 552025 filas seleccionadas
Gallego: 598375 filas seleccionadas
Total combinado: 1150400 filas


Si ya tenemos el dataset completo en un csv

In [ ]:
# df_combined = pd.read_csv('df_combined.csv', ignore_index=True)

TF-IDF

In [ ]:
# Prepara el train y test sets para trabajar con TfidfVectorizer
X_train = df_combined[df_combined['split']=='train']["text"]
y_train = df_combined[df_combined['split']=='train']['language']
X_test = df_combined[df_combined['split']=='test']["text"]
y_test = df_combined[df_combined['split']=='test']['language']

# Aplicamos TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

## Shallow learning

In [ ]:
def classify_text(X_train, y_train, X_test, y_test, feature_name="features"):
  # Logistic Regression
  lr = LogisticRegression(random_state=42).fit(X_train, y_train)
  y_pred = lr.predict(X_test)
  print(classification_report(y_test, y_pred, zero_division=0))
  print(f'The accuracy of the LogisticRegression using the {feature_name} is {round(accuracy_score(y_test, y_pred), 4)}')

  # Random Forest
  rfc = RandomForestClassifier(max_depth=2, random_state=42).fit(X_train, y_train)
  y_pred = rfc.predict(X_test)
  print(classification_report(y_test, y_pred))
  print(f'The accuracy of the RandomForestClassifier using the {feature_name} is {round(accuracy_score(y_test, y_pred), 4)}')

  # Decision Tree
  dtc = DecisionTreeClassifier().fit(X_train, y_train)
  y_pred = dtc.predict(X_test)
  print(classification_report(y_test, y_pred))
  print(f'The accuracy of the DecisionTreeClassifier using the {feature_name} is {round(accuracy_score(y_test, y_pred), 4)}')

In [ ]:
# Usar la función con TF-IDF
classify_text(X_train_tfidf, y_train, X_test_tfidf, y_test, feature_name="TF-IDF features")

              precision    recall  f1-score   support

          gl       0.99      0.99      0.99     89757
          pt       1.00      1.00      1.00    210565

    accuracy                           0.99    300322
   macro avg       0.99      0.99      0.99    300322
weighted avg       0.99      0.99      0.99    300322

The accuracy of the LogisticRegression using the TF-IDF features is 0.995


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

          gl       0.30      1.00      0.46     89757
          pt       0.00      0.00      0.00    210565

    accuracy                           0.30    300322
   macro avg       0.15      0.50      0.23    300322
weighted avg       0.09      0.30      0.14    300322

The accuracy of the RandomForestClassifier using the TF-IDF features is 0.2989
              precision    recall  f1-score   support

          gl       0.97      0.98      0.98     89757
          pt       0.99      0.99      0.99    210565

    accuracy                           0.99    300322
   macro avg       0.98      0.98      0.98    300322
weighted avg       0.99      0.99      0.99    300322

The accuracy of the DecisionTreeClassifier using the TF-IDF features is 0.9851


## Clases necesarias

Estas clases son utilizadas para manejar adecuadamente el vocabulario, vectorizer y dataset. Son las mismas que vimos en la práctica sobre CNN en clase.

In [ ]:
class Vocabulary(object):
    """Clase para procesar texto y extrar el vocabulario existente para su posterior mapeo."""

    def __init__(self, token_to_idx=None):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token
                              for token, idx in self._token_to_idx.items()}

    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index

    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary

        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token

        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        """

        return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index

        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

In [ ]:
class SequenceVocabulary(Vocabulary):
    """Extensión de la clase vocabulario para lidiar con la secuencialidad del texto."""
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx) #Llamamos al constructor padre.

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        #Usamos el vocabulario inicial y añadimos los nuevos tokens de uso especial.
        self.mask_index = self.add_token(self._mask_token) #Padding
        self.unk_index = self.add_token(self._unk_token) #Tokens desconocidos
        self.begin_seq_index = self.add_token(self._begin_seq_token) #Inicio de una frase.
        self.end_seq_index = self.add_token(self._end_seq_token) #Fin de una frase.

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token
          or the UNK index if token isn't present.

        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
              for the UNK functionality
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

In [ ]:
class LanguagesVectorizer(object):
    """ Usando el vocabulario, transforma cada token a su ID numérico y añade los tokens auxiliares necesarios (MASK, BEGIN, END...)"""
    def __init__(self, text_vocab, language_vocab):
        self.text_vocab = text_vocab #Vocabulario de todos los tokens que usamos.
        self.language_vocab = language_vocab #Vocabulario que representa las 4 categorias a las que estamos clasificando las noticias.

    def vectorize(self, text, vector_length=-1):
        """
        Args:
            text (str): the string of words separated by a space
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            the vectorized text (numpy.array)
        """
        indices = [self.text_vocab.begin_seq_index]
        indices.extend(self.text_vocab.lookup_token(token)
                       for token in text.split(" "))
        indices.append(self.text_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        #Todo 0s con longitud maxima
        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.text_vocab.mask_index

        return out_vector

    @classmethod
    def from_dataframe(cls, language_df, cutoff=5):
        """Instantiate the vectorizer from the dataset dataframe

        Args:
            language_df (pandas.DataFrame): the target dataset
            cutoff (int): frequency threshold for including in Vocabulary
        Returns:
            an instance of the LanguagesVectorizer
        """
        language_vocab = Vocabulary()
        for language in sorted(set(language_df.language)):
            language_vocab.add_token(language)

        word_counts = Counter()
        for text in language_df.text:
            for token in text.split(" "):
                if token not in string.punctuation:
                    word_counts[token] += 1

        text_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                text_vocab.add_token(word)

        return cls(text_vocab, language_vocab)

    @classmethod
    def from_serializable(cls, contents):
        text_vocab = \
            SequenceVocabulary.from_serializable(contents['text_vocab'])
        language_vocab =  \
            Vocabulary.from_serializable(contents['language_vocab'])

        return cls(text_vocab=text_vocab, language_vocab=language_vocab)

    def to_serializable(self):
        return {'text_vocab': self.text_vocab.to_serializable(),
                'language_vocab': self.language_vocab.to_serializable()}

def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [ ]:
class LanguageDataset(Dataset):
    def __init__(self, language_df, vectorizer):
        """
        Args:
            language_df (pandas.DataFrame): the dataset
            vectorizer (LanguagesVectorizer): vectorizer instatiated from dataset
        """
        self.language_df = language_df
        self._vectorizer = vectorizer

        # +1 if only using begin_seq, +2 if using both begin and end seq tokens
        measure_len = lambda context: len(context.split(" "))
        self._max_seq_length = max(map(measure_len, language_df.text)) + 2


        self.train_df = self.language_df[self.language_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.language_df[self.language_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.language_df[self.language_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

        # Class weights
        class_counts = language_df.language.value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.language_vocab.lookup_token(item[0])
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)


    @classmethod
    def load_dataset_and_make_vectorizer(cls, language_csv):
        """Load dataset and make a new vectorizer from scratch

        Args:
            surname_csv (str): location of the dataset
        Returns:
            an instance of SurnameDataset
        """
        language_df = pd.read_csv(language_csv)
        train_language_df = language_df[language_df.split=='train']
        return cls(language_df, LanguagesVectorizer.from_dataframe(train_language_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, language_csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer.
        Used in the case in the vectorizer has been cached for re-use

        Args:
            surname_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of SurnameDataset
        """
        language_df = pd.read_csv(language_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(language_csv, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file

        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of LanguagesVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return LanguagesVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json

        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets

        Args:
            index (int): the index to the data point
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]

        text_value = str(row['text'])  # Forzar conversión a cadena
        text_vector = \
            self._vectorizer.vectorize(text_value, self._max_seq_length)

        language_index = \
            self._vectorizer.language_vocab.lookup_token(row.language)

        return {'x_data': text_vector,
                'y_target': language_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset

        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size

def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## Utils

Funciones para entrenar

In [ ]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        else:
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t


            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [ ]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def download_embedding_file(language):
  fasttext.util.download_model(language, if_exists='ignore') # Descargar el modelo directamente

def get_embedding_file(filename):
    model = fasttext.load_model(filename) # Cargamos el modelo descargado
    return model

def make_embedding_matrix( fasttext_path_gl, fasttext_path_pt, words_idx_to_token, emb_dim, language):
  # Uso de un modelo u otro
  if language == "gl":
    ft = get_embedding_file(fasttext_path_gl)
  elif language == "pt":
    ft = get_embedding_file(fasttext_path_pt)
  final_embeddings = np.zeros((len(words_idx_to_token), emb_dim))
  for idx in sorted(words_idx_to_token):
      if ft.get_word_id(words_idx_to_token[idx]):
        # if the model does not have the word get_word_id returns -1, but fasttext models are able to make vectors even if a word is not in the vocabulary
        final_embeddings[idx, :] = ft.get_word_vector(words_idx_to_token[idx])
  # Cuando los embeddings ya estén borramos el modelo de fasttext de memoria
  del ft
  return final_embeddings

## CNN y Perceptron

Clase para hacer un classifier con una CNN

In [ ]:
class CNN_NLP(nn.Module):
    """An 1D Convulational Neural Network for Sentence Classification.
    Adapted from: https://www.kaggle.com/code/williamlwcliu/cnn-text-classification-pytorch"""
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=100,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=4,
                 dropout=0.5):
        """
        The constructor for CNN_NLP class.

        Args:
            pretrained_embedding (torch.Tensor): Pretrained embeddings with
                shape (vocab_size, embed_dim)
            freeze_embedding (bool): Set to False to fine-tune pretraiend
                vectors. Default: False
            vocab_size (int): Need to be specified when not pretrained word
                embeddings are not used.
            embed_dim (int): Dimension of word vectors. Need to be specified
                when pretrained word embeddings are not used. Default: 100
            filter_sizes (List[int]): List of filter sizes. Default: [3, 4, 5]
            num_filters (List[int]): List of number of filters, has the same
                length as `filter_sizes`. Default: [100, 100, 100]
            n_classes (int): Number of classes. Default: 2
            dropout (float): Dropout rate. Default: 0.5
        """

        super(CNN_NLP, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            pretrained_embeddings = torch.from_numpy(pretrained_embedding).float()
            self.emb = nn.Embedding(embedding_dim=embed_dim,
                                    num_embeddings=vocab_size,
                                    padding_idx=0,
                                    _weight=pretrained_embeddings)
        else:
            self.embed_dim = embed_dim
            self.emb = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.emb(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]

        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)

        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))

        return logits

Clase para hacer un classifier con un Perceptron de una fully connected layer

In [ ]:
class PerceptronClassifier(nn.Module):
    """Perceptron based Classifier.
    """
    def __init__(self, num_features):
        """
        Args: num_features(init): The size of input feature vector.
        """
        super(PerceptronClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features,
                            out_features=1)

    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier

        Args:
            x_in (torch.Tensor): an input data tensor
                x_in.shape should be (batch, num_features)
            apply_sigmoid (bool): a flag for the sigmoid activation
                should be false if used with the crossentropy losses.
        Returns:
            the resulting tensor. tensor.shape should be (batch,).
        """
        y_out = self.fc1(x_in).squeeze()
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
        return y_out

## Creamos un DatasetDict con un subset de los datasets originales

Cogemos un subset de 1000 lineas por dataset y los juntamos

In [ ]:
# Cargar ambos datasets
df_pt = pd.read_csv(file_path_pt)
df_gl = pd.read_csv(file_path_gl)

# Seleccionar subconjuntos de cada dataset
def select_subset(df, train_size, test_size, val_size):
    train = df[df['split'] == 'train'].head(train_size)
    test = df[df['split'] == 'test'].head(test_size)
    val = df[df['split'] == 'val'].head(val_size)
    return pd.concat([train, test, val])

# Seleccionar 700 de train, 150 de test, y 150 de val para cada idioma
df_pt_subset = select_subset(df_pt, train_size=700, test_size=150, val_size=150)
df_gl_subset = select_subset(df_gl, train_size=700, test_size=150, val_size=150)

df_combined = pd.concat([df_pt_subset, df_gl_subset], ignore_index=True)

# Verificar resultados
print(f"Portugués: {len(df_pt_subset)} filas seleccionadas")
print(f"Gallego: {len(df_gl_subset)} filas seleccionadas")
print(f"Combinados: {len(df_combined)} filas seleccionadas")

Portugués: 1000 filas seleccionadas
Gallego: 1000 filas seleccionadas
Combinados: 2000 filas seleccionadas


In [ ]:
df_combined.head()

,text,split,language
0,Deve a Ré ser condenada a pagar ao Autor a dif...,train,pt
1,"O valor equivalente a 9/12 de uma remuneração,...",train,pt
2,"O Acórdão recorrido está, salvo o devido respe...",train,pt
3,"Há, por isso, erro de julgamento quando o Trib...",train,pt
4,Ora é pacificamente aceite que na tarefa de qu...,train,pt


In [ ]:
df_combined.to_csv('df_combined_sample.csv', index=False)

## CNN with galician fasttext embeddings

In [ ]:
args = Namespace(
    # Data and Path hyper parameters
    # languages_df= '/content/drive/My Drive/4º Carrera/NLP/Proyecto/df_combined_sample.csv',  # PATH: Luis
    languages_df= '/content/drive/MyDrive/4.Curso/NLP/Proyecto/df_combined_sample.csv',  # PATH: Iker
    # vectorizer_file= '/content/drive/My Drive/4º Carrera/NLP/Proyecto/vectorizer.json',  # PATH: Luis
    vectorizer_file="/content/drive/MyDrive/4.Curso/NLP/Proyecto/vectorizer.json", # PATH: Iker
    model_state_file="model_storage/model_gl_embeddings.pth",
    # Model hyper parameters
    use_emb=True,
    use_CNN=True,
    language='gl',
    # Training hyper parameter
    seed=42,
    learning_rate=0.001,
    dropout_p=0.1,
    batch_size=128,
    num_epochs=100,
    early_stopping_criteria=2,
    # Runtime option
    cuda=True,
    catch_keyboard_interrupt=True,
)

In [ ]:
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# Load dataset and vectorizer
dataset = LanguageDataset.load_dataset_and_make_vectorizer(args.languages_df)
dataset.save_vectorizer(args.vectorizer_file) # not necessary if we have a vectorizer already
vectorizer = dataset.get_vectorizer()

if args.use_emb:
  text_vocab = vectorizer.text_vocab
  emb_gl_path = '/content/drive/MyDrive/4.Curso/NLP/Proyecto/embeddings/cc.gl.300.bin'# PATH: Iker
  # emb_gl_path = '/content/drive/MyDrive/4º Carrera/NLP/Proyecto/embeddings/cc.gl.300.bin'# PATH: Luis
  emb_pt_path = '/content/drive/MyDrive/4.Curso/NLP/Proyecto/embeddings/cc.pt.300.bin'# PATH: Iker
  # emb_pt_path = '/content/drive/MyDrive/4º Carrera/NLP/Proyecto/embeddings/cc.pt.300.bin'# PATH: Luis

  embeddings = make_embedding_matrix(emb_gl_path, emb_pt_path, text_vocab._idx_to_token, 300, 'gl')
  print("Using pre-trained fasttext embeddings")
else:
  print("Not using pre-trained embeddings")
  embeddings = None


Using CUDA: True
<Vocabulary(size=36952)>
Using pre-trained fasttext embeddings


In [ ]:
if args.use_CNN:
  classifier = CNN_NLP(pretrained_embedding=embeddings,
                        freeze_embedding=True,
                        vocab_size=len(vectorizer.text_vocab),  # Usa el número de filas de embeddings
                        embed_dim=embeddings.shape[1],   # Usa el número de columnas de embeddings
                        num_classes=len(vectorizer.language_vocab))
  print("Using CNN")

Using CNN


### Training

Training Loop

In [ ]:
args.device = torch.device("cuda" if args.cuda else "cpu")

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)

loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                           mode='min', factor=0.5,
                                           patience=1)

train_state = make_train_state(args)

summary_shown = False

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on

        dataset.set_split('train')
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            if not summary_shown:
                print(summary(classifier, input_data=[batch_dict['x_data']]))
                summary_shown = True
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)


        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred =  classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)


        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)
        print(f"In epoch {epoch_index} el vall_acc es {running_acc} y el val_loss es {running_loss}" )

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])
        print(f"Early stopping state is {train_state['early_stopping_step']}")
        if train_state['stop_early']:
            break

except KeyboardInterrupt:
    print("Exiting loop")


Layer (type:depth-idx)                   Output Shape              Param #
CNN_NLP                                  [128, 2]                  --
├─Embedding: 1-1                         [128, 10710, 300]         11,085,600
├─ModuleList: 1-2                        --                        --
│    └─Conv1d: 2-1                       [128, 100, 10708]         90,100
│    └─Conv1d: 2-2                       [128, 100, 10707]         120,100
│    └─Conv1d: 2-3                       [128, 100, 10706]         150,100
├─Dropout: 1-3                           [128, 300]                --
├─Linear: 1-4                            [128, 2]                  602
Total params: 11,446,502
Trainable params: 11,446,502
Non-trainable params: 0
Total mult-adds (G): 495.20
Input size (MB): 10.97
Forward/backward pass size (MB): 6579.30
Params size (MB): 45.79
Estimated Total Size (MB): 6636.06
In epoch 0 el vall_acc es 99.38038793103449 y el val_loss es 0.013770447666596235
Early stopping state is 0
In ep

In [ ]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset,
                                   batch_size=args.batch_size,
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_data'])

    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc
print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

<ipython-input-35-5ef9a1f24db8>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(torch.load(train_state['model_filename']))


Test loss: 0.005528026166349923;
Test Accuracy: 99.83836206896552


## CNN with portuguese fasttext embeddings

In [ ]:
# Vaciar parte de la memoria de GPU
torch.cuda.empty_cache()

In [ ]:
args = Namespace(
    # Data and Path hyper parameters
    # languages_df= '/content/drive/My Drive/4º Carrera/NLP/Proyecto/df_combined_sample.csv',  # PATH: Luis
    languages_df= '/content/drive/MyDrive/4.Curso/NLP/Proyecto/df_combined_sample.csv',  # PATH: Iker
    # vectorizer_file= '/content/drive/My Drive/4º Carrera/NLP/Proyecto/vectorizer.json',  # PATH: Luis
    vectorizer_file="/content/drive/MyDrive/4.Curso/NLP/Proyecto/vectorizer.json", # PATH: Iker
    model_state_file="model_storage/model_pt_embeddings.pth",
    # Model hyper parameters
    use_emb=True,
    use_CNN=True,
    language='pt',
    # Training hyper parameter
    seed=42,
    learning_rate=0.001,
    dropout_p=0.1,
    batch_size=128,
    num_epochs=100,
    early_stopping_criteria=2,
    # Runtime option
    cuda=True,
    catch_keyboard_interrupt=True,
)

In [ ]:
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# Load dataset and vectorizer
dataset = LanguageDataset.load_dataset_and_make_vectorizer(args.languages_df)
dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

if args.use_emb:
  text_vocab = vectorizer.text_vocab
  print(text_vocab)
  # Si no los tenemos podemos descargar los archivos de embeddings de fasttext directamente con esta funcion
  # download_embedding_file('gl')
  # download_embedding_file('pt')
  emb_gl_path = '/content/drive/MyDrive/4.Curso/NLP/Proyecto/embeddings/cc.gl.300.bin'# PATH: Iker
  # emb_gl_path = '/content/drive/MyDrive/4º Carrera/NLP/Proyecto/embeddings/cc.gl.300.bin'# PATH: Luis
  emb_pt_path = '/content/drive/MyDrive/4.Curso/NLP/Proyecto/embeddings/cc.pt.300.bin'# PATH: Iker
  # emb_pt_path = '/content/drive/MyDrive/4º Carrera/NLP/Proyecto/embeddings/cc.pt.300.bin'# PATH: Luis

  embeddings = make_embedding_matrix(emb_gl_path, emb_pt_path, text_vocab._idx_to_token, 300, 'pt')
  print("Using pre-trained fasttext embeddings")
else:
  print("Not using pre-trained embeddings")
  embeddings = None


Using CUDA: True
<Vocabulary(size=36952)>
Using pre-trained fasttext embeddings


In [ ]:
if args.use_CNN:
  classifier = CNN_NLP(pretrained_embedding=embeddings,
                        freeze_embedding=True, # Si congelamos los embeddings para que no se actualicen durante el training
                        vocab_size=len(vectorizer.text_vocab),  # Usa el número de filas de embeddings
                        embed_dim=embeddings.shape[1],   # Usa el número de columnas de embeddings
                        num_classes=len(vectorizer.language_vocab)) # 2, gallego y portugues
  print("Using CNN")

Using CNN


### Training

Training Loop

In [ ]:
args.device = torch.device("cuda" if args.cuda else "cpu")

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)

loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                           mode='min', factor=0.5,
                                           patience=1)

train_state = make_train_state(args)

summary_shown = False

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on

        dataset.set_split('train')
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            if not summary_shown:
                print(summary(classifier, input_data=[batch_dict['x_data']]))
                summary_shown = True
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)


        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset,
                                           batch_size=args.batch_size,
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred =  classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)


        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)
        print(f"In epoch {epoch_index} el vall_acc es {running_acc} y el val_loss es {running_loss}" )

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])
        print(f"Early stopping state is {train_state['early_stopping_step']}")
        if train_state['stop_early']:
            break

except KeyboardInterrupt:
    print("Exiting loop")


Layer (type:depth-idx)                   Output Shape              Param #
CNN_NLP                                  [128, 2]                  --
├─Embedding: 1-1                         [128, 10710, 300]         11,085,600
├─ModuleList: 1-2                        --                        --
│    └─Conv1d: 2-1                       [128, 100, 10708]         90,100
│    └─Conv1d: 2-2                       [128, 100, 10707]         120,100
│    └─Conv1d: 2-3                       [128, 100, 10706]         150,100
├─Dropout: 1-3                           [128, 300]                --
├─Linear: 1-4                            [128, 2]                  602
Total params: 11,446,502
Trainable params: 11,446,502
Non-trainable params: 0
Total mult-adds (G): 495.20
Input size (MB): 10.97
Forward/backward pass size (MB): 6579.30
Params size (MB): 45.79
Estimated Total Size (MB): 6636.06
In epoch 0 el vall_acc es 99.54202586206895 y el val_loss es 0.017295433995153372
Early stopping state is 0
In ep

In [ ]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset,
                                   batch_size=args.batch_size,
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_data'])

    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc
print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

<ipython-input-22-5ef9a1f24db8>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(torch.load(train_state['model_filename']))


Test loss: 0.008437231710935331;
Test Accuracy: 99.78448275862068


## Perceptron

### Clases para funcionamiento de Perceptron

Estas clases son utilizadas para manejar adecuadamente el vocabulario, vectorizer y dataset. Son las mismas que vimos en la práctica sobre Perceptron en clase.

In [ ]:
class PerceptronVocabulary(object):
    """Clase para procesar texto y extrar el vocabulario existente para su posterior mapeo.
    """
    def __init__(self, token_to_idx=None, add_unk=True, unk_token=""):
        """
        Args: token_to_idx(dict): Pre existing map of Tokens to Index.
            : add_unk(bool): A flag indicating whether to add UNK Token.
            : unk_token(string): The UNK Token to add in Vocabulary.
        """
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx:token for token,idx in self._token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token)

    def add_token(self, token):
        """Update the mapping dictionary based on the Tokens.
        Args: token: The item to add into the Vocabulary.
        Returns: index: Integer corresponding to the Token.
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index

    def add_many(self, tokens):
        """Add a list of Tokens into Vocabulary.
        Args: tokens(list): A list of string Tokens.
        Returns: indices(list): A list of indices correspoinding to the Tokens.
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the Index associated with the Token.
        Args: token(str): The Token to lookup.
        Returns: index(int): The Index correspoinding to the Token.
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the Token associated with the Index.
        Args: index(int): The Index to lookup.
        Returns: token(str): The Token correspoinding to the Index.
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary (size=%d)" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

In [ ]:
class PerceptronVectorizer(object):
    """The Vectorizer coordinates the Vocabularies and puts them to use.
    """
    def __init__(self, text_vocab, language_vocab):
        self.text_vocab = text_vocab #Vocabulario de todos los tokens que usamos.
        self.language_vocab = language_vocab #Vocabulario que representa las 4 categorias a las que estamos clasificando las noticias.

    def vectorize(self, review):
        """Create a collasped Onehot Vector for the review.
        Args: review: The review
        Returns: one_hot: The collapsed one hot Encoding.
        """
        one_hot = np.zeros(len(self.text_vocab), dtype=np.float32)
        for token in review.split(" "):
            if token not in string.punctuation:
                one_hot[self.text_vocab.lookup_token(token)] = 1
        return one_hot

    @classmethod
    def from_dataframe(cls, language_df, cutoff=5):
        """Instantiate the vectorizer from the dataset dataframe

        Args:
            language_df (pandas.DataFrame): the target dataset
            cutoff (int): frequency threshold for including in Vocabulary
        Returns:
            an instance of the LanguagesVectorizer
        """
        language_vocab = PerceptronVocabulary()
        for language in sorted(set(language_df.language)):
            language_vocab.add_token(language)

        word_counts = Counter()
        for text in language_df.text:
            for token in text.split(" "):
                if token not in string.punctuation:
                    word_counts[token] += 1

        text_vocab = PerceptronVocabulary(add_unk=True)
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                text_vocab.add_token(word)

        return cls(text_vocab, language_vocab)

    def get_text_vocab(self):
        """Returns the review Vocabulary.
        """
        return self.text_vocab

In [ ]:
class PerceptronDataset(Dataset):
    def __init__(self, language_df, vectorizer):
        """
        Args: language_df(pandas.DataFrame): The dataset.
            : vectorizer(ReviewVectorizer): Vector instantiated from dataset.
        """
        self.language_df = language_df
        self._vectorizer = vectorizer

        self.train_df = self.language_df[self.language_df.split == "train"]
        self.train_size = len(self.train_df)

        self.val_df = self.language_df[self.language_df.split == "val"]
        self.validation_size = len(self.val_df)

        self.test_df = self.language_df[self.language_df.split == "test"]
        self.test_size = len(self.test_df)

        self._lookup_dict = {"train": (self.train_df, self.train_size),
                                "val": (self.val_df, self.validation_size),
                                "test": (self.test_df, self.test_size)}
        print({"train": self.train_size,
                                "val": self.validation_size,
                                "test": self.test_size})
        self.set_split("train")

    @classmethod
    def load_dataset_and_make_vectorizer(cls, language_csv):
        """Load dataset and make new vectorizer from scratch.
        Args: language_csv: Location of the dataset.
        Returns: An instance of ReviewDataset.
        """
        language_df = pd.read_csv(language_csv)
        train_language_df = language_df[language_df.split == "train"]
        return cls(language_df, PerceptronVectorizer.from_dataframe(train_language_df))

    def save_vectorizer(self, vectorizer_filepath):
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """Returns the Vectorizer.
        """
        return self._vectorizer

    def set_split(self, split="train"):
        """Splits the dataset using a column in the DataFrame.
        Args: split(str): One of "train", "val" or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """Primary entry point of PyTorch Datasets.
        Args: index: Index of the Datapoint.
        Returns: A dictionary holding the Data point features and labels.
        """

        row = self._target_df.iloc[index]
        text_value = str(row['text'])  # Forzar conversión a cadena
        text_vector = self._vectorizer.vectorize(text_value)
        language_index = self._vectorizer.language_vocab.lookup_token(row.language)
        return {"x_data": text_vector,
                "y_target": torch.tensor(language_index, dtype=torch.float32)}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the Dataset.
        Args: batch_size(int)
        Returns: Number of batches in the Dataset.
        """
        return len(self) // batch_size

In [ ]:
def compute_accuracy(y_pred, y_target):
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).long()
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

### Training

In [ ]:
# Vaciar parte de la memoria de GPU
torch.cuda.empty_cache()

In [ ]:
args = Namespace(
    # Data and Path hyper parameters
    # languages_df= '/content/drive/My Drive/4º Carrera/NLP/Proyecto/df_combined_sample.csv',  # PATH: Luis
    languages_df= '/content/drive/MyDrive/4.Curso/NLP/Proyecto/df_combined_sample.csv',  # PATH: Iker
    # vectorizer_file= '/content/drive/My Drive/4º Carrera/NLP/Proyecto/vectorizer.json',  # PATH: Luis
    vectorizer_file="/content/drive/MyDrive/4.Curso/NLP/Proyecto/vectorizer.json", # PATH: Iker
    model_state_file="model_storage/perceptron.pth",
    # Training hyper parameter
    seed=42,
    learning_rate=0.001,
    batch_size=128,
    num_epochs=50,
    # Runtime option
    cuda=True,
)

Training Loop

In [ ]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
  """A generator function which wraps the PyTorch DataLoader.
  """
  dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                          shuffle=shuffle, drop_last=drop_last)
  for data_dict in dataloader:
    out_data_dict = {}
    for name, tensor in data_dict.items():
      out_data_dict[name] = data_dict[name].to(device)
    yield out_data_dict

In [ ]:
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# Load dataset and vectorizer
dataset = PerceptronDataset.load_dataset_and_make_vectorizer(args.languages_df)
vectorizer = dataset.get_vectorizer()
classifier = PerceptronClassifier(num_features=len(vectorizer.text_vocab))

loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

train_state = make_train_state(args)

#Empezamos el outer loop que itera N veces siendo N el número de epochs.
for epoch_index in range(args.num_epochs):
    train_state['epoch_index'] = epoch_index

    # Generamos los training batches con el train set.
    dataset.set_split('train')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.0
    running_acc = 0.0
    #Llamamos al método .train() del clasificador para indicar que el modelo está en "modo de entrenamiento" y los parámetros del modelo son mutables.
    classifier.train()
    for batch_index, batch_dict in enumerate(batch_generator):
        # Por cada minibatch ejecutamos la rutina de entrenamiento:
        # step 1. Ponemos a 0 los gradientes
        optimizer.zero_grad()
        # step 2. Calculamos el ouput del clasificador en este paso.
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 3. Calculamos el loss del clasificador en este paso.
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index + 1)
        # step 4. Usamos el loss para producir los gradientes.
        loss.backward()
        # step 5.Usamos  el optimizador para dar un paso de gradiente
        optimizer.step()
        # --------------------------------------------------
        #Calculmos el accuracy para tener una métrica extra además del loss.
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)
    print("Train loss: {:.3f} in Epoch {}".format(running_loss, epoch_index))
    print("Train accuracy: {:.3f} in Epoch {}".format(running_acc, epoch_index))
    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)
    # Iterate over val dataset

    # setup: batch generator, set loss and acc to 0, set eval mode on
    dataset.set_split('val')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()
    for batch_index, batch_dict in enumerate(batch_generator):
        # step 1. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 2. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index + 1)
        # step 3. compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)
    print("Val loss: {:.3f} in Epoch {}".format(running_loss, epoch_index))
    print("Val accuracy: {:.3f} in Epoch {}".format(running_acc, epoch_index))
    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)


torch.save(classifier.state_dict(), train_state['model_filename'])

Using CUDA: False
{'train': 17500, 'val': 3750, 'test': 3750}
Train loss: -0.605 in Epoch 0
Train accuracy: 49.822 in Epoch 0
Val loss: -1.522 in Epoch 0
Val accuracy: 50.081 in Epoch 0
Train loss: -2.303 in Epoch 1
Train accuracy: 50.017 in Epoch 1
Val loss: -3.066 in Epoch 1
Val accuracy: 50.000 in Epoch 1
Train loss: -3.868 in Epoch 2
Train accuracy: 50.023 in Epoch 2
Val loss: -4.641 in Epoch 2
Val accuracy: 50.000 in Epoch 2
Train loss: -5.489 in Epoch 3
Train accuracy: 49.989 in Epoch 3
Val loss: -6.288 in Epoch 3
Val accuracy: 49.838 in Epoch 3
Train loss: -7.143 in Epoch 4
Train accuracy: 49.983 in Epoch 4
Val loss: -7.928 in Epoch 4
Val accuracy: 50.000 in Epoch 4
Train loss: -8.823 in Epoch 5
Train accuracy: 49.989 in Epoch 5
Val loss: -9.623 in Epoch 5
Val accuracy: 49.865 in Epoch 5
Train loss: -10.522 in Epoch 6
Train accuracy: 50.006 in Epoch 6
Val loss: -11.298 in Epoch 6
Val accuracy: 49.973 in Epoch 6
Train loss: -12.237 in Epoch 7
Train accuracy: 50.006 in Epoch 7
Val

In [ ]:
# compute the loss & accuracy on the test set using the best available model
classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
loss_func = nn.CrossEntropyLoss()

dataset.set_split('test')
batch_generator = generate_batches(dataset,
                                   batch_size=args.batch_size,
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_data'])

    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc
print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

<ipython-input-26-5aa054e4da56>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(torch.load(train_state['model_filename']))


Test loss: 125523.11314655174;
Test Accuracy: 50.13469827586207
